# Assignment 2

## Initialization

In [ ]:
#@title Mount your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. You may also
# change the paths for data/logs in Arguments below.
%matplotlib inline
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@title Link your assignment folder & install requirements
#@markdown Enter the path to the assignment folder in your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. You may also
# change the paths for data/logs in Arguments below.
import sys
import os
import shutil
import warnings

folder = "" #@param {type:"string"}
!ln -Ts "$folder" /content/assignment 2> /dev/null

# Add the assignment folder to Python path
if '/content/assignment' not in sys.path:
  sys.path.insert(0, '/content/assignment')

# Install requirements
!pip install -qr /content/assignment/requirements.txt

# Check if CUDA is available
import torch
if not torch.cuda.is_available():
  warnings.warn('CUDA is not available.')

### Running on GPU
For this assignment, it will be necessary to run your experiments on GPU. To make sure the notebook is running on GPU, you can change the notebook settings with
* (EN) `Edit > Notebook Settings`
* (FR) `Modifier > Paramètres du notebook`


In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import urllib.request

from dataclasses import dataclass
from torch.utils.data import DataLoader
from tqdm import tqdm

from lstm_solution import LSTM
from gpt1_solution import MiniGPT1
from utils.wikitext2 import Wikitext2
from utils.torch_utils import seed_experiment, to_device
from utils.gpu_usage import get_gpu_memory_usage
from utils.data_utils import save_logs
from run_exp import train, evaluate

EMBEDDINGS_URL = "https://www.dropbox.com/s/g91502hubcmb4ob/embeddings.npz?dl=0"

## Public tests
Run the following cell in order to run the public tests to check to tensor shapes of the outputs of your functions.

In [2]:
!python -m unittest discover -s /content/assignment/

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/jaydan/anaconda3/envs/ift6135/lib/python3.11/unittest/__main__.py", line 18, in <module>
    main(module=None)
  File "/home/jaydan/anaconda3/envs/ift6135/lib/python3.11/unittest/main.py", line 101, in __init__
    self.parseArgs(argv)
  File "/home/jaydan/anaconda3/envs/ift6135/lib/python3.11/unittest/main.py", line 127, in parseArgs
    self._do_discovery(argv[2:])
  File "/home/jaydan/anaconda3/envs/ift6135/lib/python3.11/unittest/main.py", line 247, in _do_discovery
    self.createTests(from_discovery=True, Loader=Loader)
  File "/home/jaydan/anaconda3/envs/ift6135/lib/python3.11/unittest/main.py", line 157, in createTests
    self.test = loader.discover(self.start, self.pattern, self.top)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jaydan/anaconda3/envs/ift6135/lib/python3.11/unittest/loader.

## Experiments

Below we define a few default arguments to get you started with your experiments. You are encouraged to modify the function `main()`, as well as these arguments, to fit your needs (e.g. changing hyperparameters, the optimizer, adding regularization, adding logs).

In [ ]:
@dataclass
class Arguments:
  # Data
  data_folder: str = '/content/assignment/data'
  batch_size: int = 16

  # Model
  model: str = 'lstm'  # [lstm, gpt1]
  embeddings: str = '/content/assignment/data/embeddings.npz'
  layers: int = 1

  # Optimization
  optimizer: str = 'adamw'  # [sgd, momentum, adam, adamw]
  epochs: int = 10
  lr: float = 1e-3
  momentum: float = 0.9
  weight_decay: float = 5e-4

  # Experiment
  exp_id: str = 'debug'
  log: bool = True
  log_dir: str = '/content/assignment/logs'
  seed: int = 42

  # Miscellaneous
  num_workers: int = 2
  device: str = 'cuda'
  progress_bar: bool = False
  print_every: int = 10

The 12 configurations you need to run in Problem 3. Be careful that there is no discrepency between the configurations defined in `run_exp.py` and the ones below. In case there is a difference, the version from `run_exp.py` should be considered the ones to run.

In [ ]:
# Note: if there is any discrepency with the configurations in run_exp.py, the
# version from run_exp.py should be the ones to use in Problem 3.
configs = {
  1: Arguments(model='lstm', layers=1, batch_size=16, log=True, epochs=10, optimizer='adam', exp_id='1'),
  2: Arguments(model='lstm', layers=1, batch_size=16, log=True, epochs=10, optimizer='adamw', exp_id='2'),
  3: Arguments(model='lstm', layers=1, batch_size=16, log=True, epochs=10, optimizer='sgd', exp_id='3'),
  4: Arguments(model='lstm', layers=1, batch_size=16, log=True, epochs=10, optimizer='momentum', exp_id='4'),

  5: Arguments(model='gpt1', layers=1, batch_size=16, log=True, epochs=10, optimizer='adam', exp_id='5'),
  6: Arguments(model='gpt1', layers=1, batch_size=16, log=True, epochs=10, optimizer='adamw', exp_id='6'),
  7: Arguments(model='gpt1', layers=1, batch_size=16, log=True, epochs=10, optimizer='sgd', exp_id='7'),
  8: Arguments(model='gpt1', layers=1, batch_size=16, log=True, epochs=10, optimizer='momentum', exp_id='8'),

  9: Arguments(model='lstm', layers=2, batch_size=16, log=True, epochs=10, optimizer='adamw', exp_id='9'),
  10: Arguments(model='lstm', layers=4, batch_size=16, log=True, epochs=10, optimizer='adamw', exp_id='10'),
  11: Arguments(model='gpt1', layers=2, batch_size=16, log=True, epochs=10, optimizer='adamw', exp_id='11'),
  12: Arguments(model='gpt1', layers=4, batch_size=16, log=True, epochs=10, optimizer='adamw', exp_id='12'),
}

In [ ]:
def main(args):
  
  torch.cuda.empty_cache()
  
  # Seed the experiment, for repeatability
  seed_experiment(args.seed)

  # Dataloaders
  train_dataset = Wikitext2(args.data_folder, split="train")
  train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    shuffle=True,
    num_workers=args.num_workers,
  )

  valid_dataset = Wikitext2(args.data_folder, split="validation")
  valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=args.num_workers,
  )

  test_dataset = Wikitext2(args.data_folder, split="test")
  test_dataloader = DataLoader(
    test_dataset,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=args.num_workers,
  )

  # Download the embeddings
  if not os.path.isfile(args.embeddings):
    print("Downloading embeddings...")
    urllib.request.urlretrieve(EMBEDDINGS_URL, args.embeddings)

  # Model
  if args.model == "lstm":
    model = LSTM.load_embeddings_from(
      args.embeddings, hidden_size=512, num_layers=args.layers
    )
  elif args.model == "gpt1":
    model = MiniGPT1.load_embeddings_from(
      args.embeddings, num_layers=args.layers
    )
  else:
    raise ValueError("Unknown model {0}".format(args.model))
  model.to(args.device)

  # Optimizer
  if args.optimizer == "adamw":
    optimizer = optim.AdamW(
      model.parameters(), lr=args.lr, weight_decay=args.weight_decay
    )
  elif args.optimizer == "adam":
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
  elif args.optimizer == "sgd":
    optimizer = optim.SGD(
      model.parameters(), lr=args.lr, weight_decay=args.weight_decay
    )
  elif args.optimizer == "momentum":
    optimizer = optim.SGD(
      model.parameters(),
      lr=args.lr,
      momentum=args.momentum,
      weight_decay=args.weight_decay,
    )

  print(
    f"Initialized {args.model.upper()} model with {sum(p.numel() for p in model.parameters())} "
    f"total parameters, of which {sum(p.numel() for p in model.parameters() if p.requires_grad)} are learnable."
  )

  train_losses, valid_losses = [], []
  train_ppls, valid_ppls = [], []
  train_times, valid_times = [], []
  gpu_memories = []
  for epoch in range(args.epochs):

    tqdm.write(f"====== Epoch {epoch} ======>")
    
    mem_before = get_gpu_memory_usage()

    loss, ppl, wall_time = train(epoch, model, train_dataloader, optimizer, args)
    train_losses.append(loss)
    train_ppls.append(ppl)
    train_times.append(wall_time)

    loss, ppl, wall_time = evaluate(epoch, model, valid_dataloader, args)
    valid_losses.append(loss)
    valid_ppls.append(ppl)
    valid_times.append(wall_time)
    
    mem_after = get_gpu_memory_usage()
    avg_memory = (mem_before + mem_after) / 2
    gpu_memories.append(avg_memory)

  test_loss, test_ppl, test_time = evaluate(
    epoch, model, test_dataloader, args, mode="test"
  )

  avg_steady_state_memory = sum(gpu_memories) / len(gpu_memories)
  print(f"Average steady-state GPU memory usage: {avg_steady_state_memory:.2f} MB")  
  
  print(f"===== Best validation perplexity: {min(valid_ppls):.3f} =====>")

  return (
    train_losses,
    train_ppls,
    train_times,
    valid_losses,
    valid_ppls,
    valid_times,
    test_loss,
    test_ppl,
    test_time,
    avg_steady_state_memory
  )

In [ ]:
args = configs[1]  # Run the first configuration
logs = main(args)
if args.log:
  save_logs(args, *logs)

In [ ]:
for i in range(1, 13):
  args = configs[i]
  logs = main(args)
  if args.log:
    save_logs(args, *logs)

In [8]:
import json
import os
import matplotlib.pyplot as plt

def load_json(filepath):
    with open(filepath, 'r') as f:
        return json.load(f)

def load_txt(filepath):
    with open(filepath, 'r') as f:
        return [float(line.strip()) for line in f]
    
def save_plot(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return path

In [19]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt

base_dir = "logs/"
save_folder = "plots/"

def plot_for_experiment(exp_id):
    
    # Load data
    args_path = os.path.join(base_dir, str(exp_id), "args.json")
    with open(args_path, 'r') as f:
        args = json.load(f)

    train_loss = np.loadtxt(os.path.join(base_dir, str(exp_id), "train_loss.txt"))
    valid_loss = np.loadtxt(os.path.join(base_dir, str(exp_id), "valid_loss.txt"))
    train_ppl = np.loadtxt(os.path.join(base_dir, str(exp_id), "train_ppl.txt"))
    valid_ppl = np.loadtxt(os.path.join(base_dir, str(exp_id), "valid_ppl.txt"))
    train_time = np.cumsum(np.loadtxt(os.path.join(base_dir, str(exp_id), "train_time.txt")))
    valid_time = np.cumsum(np.loadtxt(os.path.join(base_dir, str(exp_id), "valid_time.txt")))

    caption = (
        f"Model: {args['model']}, Layers: {args['layers']}, Optimizer: {args['optimizer']}, "
        f"Learning Rate: {args['lr']}, Momentum: {args['momentum']}, Weight Decay: {args['weight_decay']}, "
        f"Batch Size: {args['batch_size']}"
    )

    # Plots
    fig, axs = plt.subplots(3, 2, figsize=(15, 20))
    fig.suptitle(f"Experiment {exp_id}'s logs in graphs", fontsize=24)
    fig.text(0.5, 0.04, caption, ha='center', fontsize=16)

    # Training loss over time
    axs[0, 0].plot(train_time, train_loss, label='Train Loss')
    axs[0, 0].set_title('Training Loss Over Time')
    axs[0, 0].set_xlabel('Time (s)')
    axs[0, 0].set_ylabel('Loss')
    axs[0, 0].legend()

    # Validation loss over time
    axs[0, 1].plot(valid_time, valid_loss, label='Validation Loss', color='r')
    axs[0, 1].set_title('Validation Loss Over Time')
    axs[0, 1].set_xlabel('Time (s)')
    axs[0, 1].set_ylabel('Loss')
    axs[0, 1].legend()

    # Training perplexity over time
    axs[1, 0].plot(train_time, train_ppl, label='Train Perplexity')
    axs[1, 0].set_title('Training Perplexity Over Time')
    axs[1, 0].set_xlabel('Time (s)')
    axs[1, 0].set_ylabel('Perplexity')
    axs[1, 0].legend()

    # Validation perplexity over time
    axs[1, 1].plot(valid_time, valid_ppl, label='Validation Perplexity', color='r')
    axs[1, 1].set_title('Validation Perplexity Over Time')
    axs[1, 1].set_xlabel('Time (s)')
    axs[1, 1].set_ylabel('Perplexity')
    axs[1, 1].legend()

    # Training and Validation loss over epoch
    epochs = list(range(1, len(train_loss)+1))
    axs[2, 0].plot(epochs, train_loss, label='Train Loss')
    axs[2, 0].plot(epochs, valid_loss, label='Validation Loss', linestyle='--')
    axs[2, 0].set_title('Training and Validation Loss Over Epoch')
    axs[2, 0].set_xlabel('Epoch')
    axs[2, 0].set_ylabel('Loss')
    axs[2, 0].legend()

    # Training and Validation perplexity over epoch
    axs[2, 1].plot(epochs, train_ppl, label='Train Perplexity')
    axs[2, 1].plot(epochs, valid_ppl, label='Validation Perplexity', linestyle='--')
    axs[2, 1].set_title('Training and Validation Perplexity Over Epoch')
    axs[2, 1].set_xlabel('Epoch')
    axs[2, 1].set_ylabel('Perplexity')
    axs[2, 1].legend()

    # save_path = os.path.join(c, str(exp_id))
    os.makedirs(save_folder, exist_ok=True)
    fig.savefig(os.path.join(save_folder, f"experiment_{exp_id}.png"))
    plt.close(fig)

for exp_id in range(1, 13):
    plot_for_experiment(exp_id)

In [26]:
import os
import json

base_path = "logs"

results = []

for exp_id in list(range(1, 13)) :
    exp_path = os.path.join(base_path, str(exp_id))
    
    # Load args.json to get model configurations
    with open(os.path.join(exp_path, "args.json"), 'r') as f:
        args = json.load(f)

    # Load validation perplexities
    with open(os.path.join(exp_path, "valid_ppl.txt"), 'r') as f:
        valid_ppls = [float(line.strip()) for line in f.readlines()]

    # Load training perplexities
    with open(os.path.join(exp_path, "train_ppl.txt"), 'r') as f:
        train_ppls = [float(line.strip()) for line in f.readlines()]

    best_epoch = valid_ppls.index(min(valid_ppls))

    result = {
        "Experiment Number": exp_id,
        "Architecture": args["model"],
        "Number of Layers": args["layers"],
        "Optimizer": args["optimizer"],
        "Best Validation Perplexity": round(valid_ppls[best_epoch], 5),
        "Training Perplexity at Best Epoch": round(train_ppls[best_epoch],5),
        "best_epoch": best_epoch,
    }
    results.append(result)

# Sort the results based on the given criteria
results.sort(key=lambda x: (x["Architecture"], x["Number of Layers"], x["Optimizer"]))

for res in results:
    print(res)


{'Experiment Number': 5, 'Architecture': 'gpt1', 'Number of Layers': 1, 'Optimizer': 'adam', 'Best Validation Perplexity': 1.112, 'Training Perplexity at Best Epoch': 1.09683, 'best_epoch': 3}
{'Experiment Number': 6, 'Architecture': 'gpt1', 'Number of Layers': 1, 'Optimizer': 'adamw', 'Best Validation Perplexity': 1.11202, 'Training Perplexity at Best Epoch': 1.09685, 'best_epoch': 3}
{'Experiment Number': 8, 'Architecture': 'gpt1', 'Number of Layers': 1, 'Optimizer': 'momentum', 'Best Validation Perplexity': 669.23708, 'Training Perplexity at Best Epoch': 702.54616, 'best_epoch': 9}
{'Experiment Number': 7, 'Architecture': 'gpt1', 'Number of Layers': 1, 'Optimizer': 'sgd', 'Best Validation Perplexity': 1422.62489, 'Training Perplexity at Best Epoch': 1528.48684, 'best_epoch': 9}
{'Experiment Number': 11, 'Architecture': 'gpt1', 'Number of Layers': 2, 'Optimizer': 'adamw', 'Best Validation Perplexity': 1.35718, 'Training Perplexity at Best Epoch': 1.38032, 'best_epoch': 7}
{'Experimen

In [27]:
import os
import json

def fetch_experiment_data(exp_id):
    data = {}
    with open(f'logs/{exp_id}/args.json', 'r') as f:
        args = json.load(f)
        data['optimizer'] = args['optimizer']
        data['architecture'] = args['model']
        data['layers'] = args['layers']

    with open(f'logs/{exp_id}/train_time.txt', 'r') as f:
        train_times = [float(line.strip()) for line in f.readlines()]
        data['total_train_time'] = sum(train_times)

    with open(f'logs/{exp_id}/valid_ppl.txt', 'r') as f:
        valid_ppls = [float(line.strip()) for line in f.readlines()]
        data['best_valid_ppl'] = min(valid_ppls)

    return data

experiments = [fetch_experiment_data(exp_id) for exp_id in range(1, 13)]

# Sorting by wall-clock time
sorted_by_time = sorted(experiments, key=lambda x: x['total_train_time'])
fastest_config = sorted_by_time[0]
print(f"Fastest configuration based on wall-clock time: {fastest_config}")

# Sorting by generalization performance
sorted_by_perf = sorted(experiments, key=lambda x: x['best_valid_ppl'])
best_generalizing_config = sorted_by_perf[0]
print(f"Best configuration based on generalization performance: {best_generalizing_config}")



Fastest configuration based on wall-clock time: {'optimizer': 'sgd', 'architecture': 'gpt1', 'layers': 1, 'total_train_time': 388.8509449958801, 'best_valid_ppl': 1422.624893995365}
Best configuration based on generalization performance: {'optimizer': 'adam', 'architecture': 'gpt1', 'layers': 1, 'total_train_time': 392.8220703601837, 'best_valid_ppl': 1.112004132695749}


In [35]:
def fetch_experiment_data(exp_id):
    with open(f'logs/{exp_id}/train_time.txt', 'r') as f:
        train_times = [float(line.strip()) for line in f.readlines()]
        total_train_time = sum(train_times)

    with open(f'logs/{exp_id}/valid_ppl.txt', 'r') as f:
        valid_ppls = [float(line.strip()) for line in f.readlines()]
        best_valid_ppl = min(valid_ppls)
    
    return round(total_train_time,2), round(best_valid_ppl,4), exp_id

time_and_ppl = [fetch_experiment_data(exp_id) for exp_id in range(1, 13)]
for total_train_time, best_valid_ppl, exp_id in time_and_ppl:
    print(f"Experiment {exp_id}: Total Training Time = {total_train_time} seconds, Best Validation Perplexity = {best_valid_ppl}")
    
print("\n\nSorted by Total Training Time:")
sorted_by_time = sorted(time_and_ppl, key=lambda x: x[0])
for total_train_time, best_valid_ppl, exp_id in sorted_by_time:
    print(f"Experiment {exp_id}: Total Training Time = {total_train_time} seconds, Best Validation Perplexity = {best_valid_ppl}")
    


Experiment 1: Total Training Time = 393.91 seconds, Best Validation Perplexity = 144.2296
Experiment 2: Total Training Time = 392.3 seconds, Best Validation Perplexity = 145.1958
Experiment 3: Total Training Time = 389.61 seconds, Best Validation Perplexity = 2173.3796
Experiment 4: Total Training Time = 390.66 seconds, Best Validation Perplexity = 1654.2252
Experiment 5: Total Training Time = 392.82 seconds, Best Validation Perplexity = 1.112
Experiment 6: Total Training Time = 393.68 seconds, Best Validation Perplexity = 1.112
Experiment 7: Total Training Time = 388.85 seconds, Best Validation Perplexity = 1422.6249
Experiment 8: Total Training Time = 390.02 seconds, Best Validation Perplexity = 669.2371
Experiment 9: Total Training Time = 502.19 seconds, Best Validation Perplexity = 141.3318
Experiment 10: Total Training Time = 644.15 seconds, Best Validation Perplexity = 159.6651
Experiment 11: Total Training Time = 500.55 seconds, Best Validation Perplexity = 1.3572
Experiment 12:

In [38]:
import numpy as np

experiments = {
    1: {"optimizer": "adam", "weight_decay": 0.0, "momentum": 0.0, "train_time": 393.9062, "val_ppl": 144.2296},
    2: {"optimizer": "adamw", "weight_decay": 5e-4, "momentum": 0.0, "train_time": 392.297, "val_ppl": 145.1958},
    3: {"optimizer": "sgd", "weight_decay": 5e-4, "momentum": 0.0, "train_time": 389.6111, "val_ppl": 2173.3796},
    4: {"optimizer": "momentum", "weight_decay": 5e-4, "momentum": 0.9, "train_time": 390.6608, "val_ppl": 1654.2252},
    5: {"optimizer": "adam", "weight_decay": 0.0, "momentum": 0.0, "train_time": 392.8221, "val_ppl": 1.112},
    6: {"optimizer": "adamw", "weight_decay": 5e-4, "momentum": 0.0, "train_time": 393.6791, "val_ppl": 1.112},
    7: {"optimizer": "sgd", "weight_decay": 5e-4, "momentum": 0.0, "train_time": 388.8509, "val_ppl": 1422.6249},
    8: {"optimizer": "momentum", "weight_decay": 5e-4, "momentum": 0.9, "train_time": 390.0189, "val_ppl": 669.2371}
}

# Group by optimizer
grouped_results = {}
for exp, data in experiments.items():
    optimizer = data["optimizer"]
    if optimizer not in grouped_results:
        grouped_results[optimizer] = {"train_times": [], "val_ppls": []}
    grouped_results[optimizer]["train_times"].append(data["train_time"])
    grouped_results[optimizer]["val_ppls"].append(data["val_ppl"])

for optimizer, data in grouped_results.items():
    mean_time = np.mean(data["train_times"])
    mean_ppl = np.mean(data["val_ppls"])
    print(f"Optimizer: {optimizer}")
    print(f"Average Training Time: {mean_time:.2f} seconds")
    print(f"Average Validation Perplexity: {mean_ppl:.2f}")
    print("-" * 40)


Optimizer: adam
Average Training Time: 393.36 seconds
Average Validation Perplexity: 72.67
----------------------------------------
Optimizer: adamw
Average Training Time: 392.99 seconds
Average Validation Perplexity: 73.15
----------------------------------------
Optimizer: sgd
Average Training Time: 389.23 seconds
Average Validation Perplexity: 1798.00
----------------------------------------
Optimizer: momentum
Average Training Time: 390.34 seconds
Average Validation Perplexity: 1161.73
----------------------------------------


In [39]:
import os

base_dir = "logs"

gpu_mem_usage = {}
for exp_id in range(1, 13):
    file_path = os.path.join(base_dir, str(exp_id), "avg_steady_state_memory.txt")
    
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            # Read the memory usage, convert to float, and store it
            gpu_mem_usage[exp_id] = float(f.read().strip())

for exp_id, mem in gpu_mem_usage.items():
    print(f"Experiment {exp_id}: Average GPU Memory Usage = {mem} MB")


Experiment 1: Average GPU Memory Usage = 4457.7 MB
Experiment 2: Average GPU Memory Usage = 4630.0 MB
Experiment 3: Average GPU Memory Usage = 4630.0 MB
Experiment 4: Average GPU Memory Usage = 4630.0 MB
Experiment 5: Average GPU Memory Usage = 4630.0 MB
Experiment 6: Average GPU Memory Usage = 4630.0 MB
Experiment 7: Average GPU Memory Usage = 4630.0 MB
Experiment 8: Average GPU Memory Usage = 4630.0 MB
Experiment 9: Average GPU Memory Usage = 4630.0 MB
Experiment 10: Average GPU Memory Usage = 4630.0 MB
Experiment 11: Average GPU Memory Usage = 5232.3 MB
Experiment 12: Average GPU Memory Usage = 5866.3 MB
